![Henry Logo](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Routing y Orquestacion simple (sin multi-agent complejo)

## Objetivo
Mostrar un patron pragmatico: un router selecciona el dominio correcto y delega en un Agentic RAG por dominio.

Esto evita sobredisenar un sistema multi-agente cuando el problema real es enrutamiento semantico + retrieval robusto.

In [1]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.vector_store_lab import build_index_from_json
from scripts.rag_pipelines import AgenticRAG, HeroRouterOrchestrator

OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

/Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batman_db, _, batman_index_stats, _ = build_index_from_json(
    json_path=ROOT / 'data' / 'batman_comics.json',
    persist_dir=OUTPUTS_DIR / 'chroma_router_batman',
    collection_name='router_batman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

spiderman_db, _, spiderman_index_stats, _ = build_index_from_json(
    json_path=ROOT / 'data' / 'spiderman_comics.json',
    persist_dir=OUTPUTS_DIR / 'chroma_router_spiderman',
    collection_name='router_spiderman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Batman index:', batman_index_stats)
print('Spider-Man index:', spiderman_index_stats)

Batman index: {'collection': 'router_batman', 'indexed_chunks': 38, 'embedding_provider': 'local:hash-embedding'}
Spider-Man index: {'collection': 'router_spiderman', 'indexed_chunks': 44, 'embedding_provider': 'local:hash-embedding'}


In [3]:
batman_pipeline = AgenticRAG(
    vector_db=batman_db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=6,
)

spiderman_pipeline = AgenticRAG(
    vector_db=spiderman_db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=6,
)

router = HeroRouterOrchestrator(
    hero_pipelines={
        'batman': batman_pipeline,
        'spiderman': spiderman_pipeline,
    },
    default_hero='batman',
)

print('Router ready.')

Router ready.


## Marco teorico: Routing heuristico vs semantico

### Por que routing basado en keywords (y no en embeddings)

El `HeroRouterOrchestrator` usa un router **heuristico** que busca keywords en la query (`"spider"`, `"joker"`, `"gotham"`, etc.) para decidir a que pipeline delegar. Esta decision tiene trade-offs claros:

| Aspecto | Heuristico (keywords) | Semantico (embeddings) |
|---|---|---|
| **Simplicidad** | Reglas explicitas, faciles de auditar | Requiere modelo de embedding + threshold de decision |
| **Determinismo** | Mismo input → misma ruta, siempre | Puede variar con actualizaciones del modelo de embedding |
| **Auditabilidad** | Se puede inspeccionar exactamente *por que* se ruteo a un dominio | El embedding es una caja negra — no explica la decision |
| **Velocidad** | O(n) con n = numero de keywords, negligible | Requiere un forward pass del embedding model |
| **Queries ambiguas** | Falla predeciblemente (va al default) | Puede capturar intenciones implicitas |

Para 2-10 dominios bien definidos (como este caso con Batman y Spider-Man), el routing heuristico es la opcion pragmatica correcta.

### Escalabilidad del patron

- **2-10 dominios**: routing por keywords funciona bien. Las reglas son manejables y el costo de mantenimiento es bajo.
- **10-50 dominios**: las reglas empiezan a superponerse y volverse fragiles. Se recomienda un clasificador de texto ligero (como un modelo de zero-shot classification o un embedding + nearest centroid).
- **50+ dominios**: se necesita routing semantico completo con un embedding de clasificacion entrenado o fine-tuneado.

### Limitaciones explicitas del router actual

Consideremos la query: *"Compara a Joker con Green Goblin en terminos de dano psicologico."*

El router evalua keywords en orden de prioridad:
1. Primero busca `"spider"`, `"peter"`, `"venom"` → no encuentra → no va a Spider-Man.
2. Luego busca `"batman"`, `"gotham"`, `"joker"`, `"wayne"` → encuentra `"joker"` → **rutea a Batman**.

Resultado: la query es cross-domain (involucra personajes de ambos universos) pero se rutea a Batman porque `"joker"` aparece antes en la evaluacion. El pipeline de Batman no tiene informacion sobre Green Goblin, asi que la respuesta sera parcial.

**Esto no es un bug, es una limitacion arquitectonica del routing heuristico.** Para queries cross-domain, se necesita un patron de fan-out (consultar ambos pipelines y fusionar resultados) o un agente de sintesis dedicado.

## Consultas de prueba

Evaluamos mezcla de consultas directas y ambiguas para observar comportamiento de routing.

In [4]:
queries = [
    'Como se transforma Bruce Wayne en Batman durante Year One?',
    'Explica la filosofia de responsabilidad de Peter Parker.',
    'Compara a Joker con Green Goblin en terminos de dano psicologico.',
    'Que rol estrategico cumple Batman en la Liga de la Justicia?',
    'Que aprendizaje deja la Saga del Clon para identidad y autenticidad?',
]

rows = []
for query in queries:
    result = router.run(query)
    rows.append(result)

routing_df = pd.DataFrame(rows)
routing_df

,selected_hero,query,answer,route,pipeline,latency_seconds,groundedness,steps
0,batman,Como se transforma Bruce Wayne en Batman duran...,Respuesta local fallback (sin llamada a OpenAI...,lore,agentic_rag,0.0019,0.8871,"[route, rewrite_query, retrieve, generate]"
1,spiderman,Explica la filosofia de responsabilidad de Pet...,Respuesta local fallback (sin llamada a OpenAI...,lore,agentic_rag,0.0014,0.9100,"[route, rewrite_query, retrieve, generate]"
2,batman,Compara a Joker con Green Goblin en terminos d...,Respuesta local fallback (sin llamada a OpenAI...,villains,agentic_rag,0.0007,0.8750,"[route, rewrite_query, retrieve, generate]"
3,batman,Que rol estrategico cumple Batman en la Liga d...,Respuesta local fallback (sin llamada a OpenAI...,strategy,agentic_rag,0.0007,0.8929,"[route, rewrite_query, retrieve, generate]"
4,batman,Que aprendizaje deja la Saga del Clon para ide...,Respuesta local fallback (sin llamada a OpenAI...,lore,agentic_rag,0.0006,0.8491,"[route, rewrite_query, retrieve, generate]"


In [5]:
summary = (
    routing_df.groupby('selected_hero', as_index=False)
    .agg(
        queries=('query', 'count'),
        avg_latency=('latency_seconds', 'mean'),
        avg_groundedness=('groundedness', 'mean'),
    )
    .round(4)
)
summary

,selected_hero,queries,avg_latency,avg_groundedness
0,batman,4,0.0010,0.876
1,spiderman,1,0.0014,0.910


In [6]:
csv_path = OUTPUTS_DIR / 'routing_orchestration_results.csv'
routing_df.to_csv(csv_path, index=False)
print(f'Saved: {csv_path}')
routing_df[['query', 'selected_hero', 'route', 'pipeline', 'groundedness']]

Saved: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/routing_orchestration_results.csv


,query,selected_hero,route,pipeline,groundedness
0,Como se transforma Bruce Wayne en Batman duran...,batman,lore,agentic_rag,0.8871
1,Explica la filosofia de responsabilidad de Pet...,spiderman,lore,agentic_rag,0.9100
2,Compara a Joker con Green Goblin en terminos d...,batman,villains,agentic_rag,0.8750
3,Que rol estrategico cumple Batman en la Liga d...,batman,strategy,agentic_rag,0.8929
4,Que aprendizaje deja la Saga del Clon para ide...,batman,lore,agentic_rag,0.8491


## Cierre didactico

### Conceptos clave de esta notebook

- **Routing heuristico es suficiente para dominios bien separados**: no necesitas embeddings ni clasificadores cuando las keywords del dominio son discriminativas y los dominios no se superponen.
- **El default hero importa**: cuando ninguna keyword matchea, la query va al dominio default. La eleccion del default debe reflejar la distribucion esperada de consultas en produccion.
- **La evaluacion de rutas es tan importante como la evaluacion de respuestas**: un retrieval perfecto no sirve si la query fue al dominio equivocado. Siempre incluye el campo `selected_hero` (o equivalente) en tus logs de produccion.
- **Queries cross-domain son el talon de Aquiles del routing simple**: si tu caso de uso tiene muchas queries que cruzan dominios, necesitas un patron mas sofisticado (fan-out, agente de sintesis, o routing semantico).

### Conexion con la siguiente notebook

En la siguiente notebook implementamos un patron **Agent2Agent** donde dos agentes especializados (Retriever y Synthesizer) colaboran via un orquestador. Esto introduce la idea de separacion de responsabilidades a nivel de agentes, con un grounding check como gate de calidad.